In [1]:
import pandas as pd
import json

In [66]:
data = pd.read_csv('../data/DM_DOCENTE.CSV', sep = "|", encoding = "latin-1")

In [67]:
data.shape

(397893, 41)

In [78]:
data = data['ID_DOCENTE'].unique()

In [81]:
data.shape

(352406,)

In [59]:
data.shape

(1, 112)

In [60]:
data.head(5)

,NU_ANO_CENSO,CO_IES,TP_CATEGORIA_ADMINISTRATIVA,TP_ORGANIZACAO_ACADEMICA,CO_LOCAL_OFERTA,CO_UF,CO_MUNICIPIO,IN_CAPITAL,CO_CURSO,NO_CURSO,...,QT_VAGAS_ANUAL_VESPERTINO,QT_VAGAS_ANUAL_NOTURNO,QT_VAGAS_ANUAL_EAD,QT_MATRICULA_TOTAL,QT_CONCLUINTE_TOTAL,QT_INGRESSO_TOTAL,QT_INGRESSO_VAGA_NOVA,QT_INGRESSO_PROCESSO_SELETIVO,QT_VAGA_TOTAL,QT_INSCRITO_TOTAL
18961,2018,1041,5,1,690748.0,43.0,4311403.0,0.0,5296,LETRAS,...,NaN,NaN,NaN,20,12,2,0,NaN,22,20
